In [39]:
import sys
import math
import pickle
import numpy as np

k = 10
n = 3
m = 2

sys.setrecursionlimit(1500)

In [40]:
def inputDigit(name="digitdata/trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image

In [41]:
digitImage = inputDigit()

In [42]:
import collections

def readRough(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels


In [43]:
label = readRough()
print (len(label))
#print sorted(collections.Counter(label).items())
class_ = sorted(collections.Counter(label).items())
#print (class_)
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

5000


In [44]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

digitImage = inputDigit()
training=np.zeros(shape=(10, 2**(n*m), 28-n+1, 28-m+1))

count = 0
for i in range(5000):                  # for each image
    for row in range(28-n+1):
        for col in range(28-m+1):      # for each feature
            
            feature_val = 0
            for y in range(n):
                for x in range(m):      # for each pixel in feature (overlapping)
                    # calculate feature_val
                    if digitImage[i*28+row+y][col+x] != ' ':
                        feature_val += 2**(2*y+x)
            training[int(label[i])][feature_val][row][col] += 1

for i in range(10):
    training[i] = (training[i] + k) / (class_[i][1] + k * 2)

print(training[0][0].shape)
#print(training1[0][0])



(26, 27)


In [9]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

test_rough = inputDigit(name = "digitdata/testimages")
answer = np.zeros(1000)


for i in range(1000):                  # for each image
    test_image = np.zeros(shape=(28-n+1, 28-m+1))
    for row in range(28-n+1):
        for col in range(28-m+1):      # for each feature
            
            feature_val = 0
            for y in range(n):
                for x in range(m):      # for each pixel in feature (overlapping)
                    # calculate feature_val
                    if test_rough[i*28+row+y][col+x] != ' ':
                        feature_val += 2**(2*y+x)
            test_image[row][col] = feature_val
       
    posteriori = np.zeros(10)
    for class_num in range(10):
        posteriori[class_num] = math.log(class_[class_num][1])
        for row in range(28):
            for col in range(28):
                if test_image[row][col] == 0:
                    posteriori[class_num] += training0[class_num][row][col]
                else:
                    posteriori[class_num] += training1[class_num][row][col]    
    answer[i] = np.argmax(posteriori)
    
#print(answer)



In [10]:
testlabels = readRough("digitdata/testlabels")


In [24]:
def confusion_matrix():
    num_each_class = np.zeros(10)
    conf_matrix = np.zeros(shape=(10,10))
    for i in range(1000):
        conf_matrix[int(testlabels[i])][int(answer[i])] += 1
        num_each_class[int(testlabels[i])] += 1
            
    for i in range(10):
        for j in range(10):
            conf_matrix[i][j] /= num_each_class[i]
    
    return conf_matrix

In [25]:
conf_matrix = confusion_matrix()
conf_matrix = np.around(conf_matrix, 2)
for row in conf_matrix:
    print(row)

[ 0.78  0.    0.    0.01  0.01  0.01  0.12  0.    0.02  0.04]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.02  0.58  0.24  0.03  0.01  0.    0.06  0.04  0.02  0.  ]
[ 0.    0.42  0.    0.49  0.    0.    0.01  0.06  0.01  0.01]
[ 0.    0.19  0.    0.    0.6   0.    0.03  0.    0.    0.19]
[ 0.03  0.35  0.    0.23  0.04  0.1   0.04  0.03  0.03  0.14]
[ 0.    0.31  0.    0.    0.07  0.    0.63  0.    0.    0.  ]
[ 0.    0.32  0.    0.    0.02  0.    0.    0.57  0.01  0.08]
[ 0.02  0.47  0.    0.06  0.01  0.    0.04  0.02  0.24  0.15]
[ 0.01  0.12  0.    0.01  0.04  0.    0.    0.01  0.01  0.8 ]
